In [ ]:
from pybis import Openbis
import json
import ipywidgets as widgets
from IPython.display import display, Javascript, HTML, clear_output
from datetime import datetime

In [ ]:
# Global variables
OPENBIS_SESSION = None
RAW_MATERIALS_TYPES = ["CRYSTAL", "WAFER_SUBSTRATE", "2D_LAYERED_MATERIAL", "MOLECULE"]
PROCESS_SAMPLE_TYPES = ["SPUTTERING", "ANNEALING", "DEPOSITION"]

In [ ]:
# Widgets
samples_dropdown = widgets.Dropdown(description='Sample', disabled=False, layout = widgets.Layout(width = '400px'))
samples_dropdown.style = {'description_width': '110px'}
sample_details_textbox = widgets.Textarea(value = '', description = '', disabled = True, layout = widgets.Layout(width = '600px', height = '300px'))
sample_metadata_boxes = widgets.HBox([samples_dropdown, sample_details_textbox])

instruments_dropdown = widgets.Dropdown(description='Instrument', disabled=False, layout = widgets.Layout(width = '993px'))
instruments_dropdown.style = {'description_width': '110px'}

measurements_files_import = widgets.FileUpload(accept = '.sxm, .dat', multiple = True)
measurements_files_import_output = widgets.Output()

upload_measurements_button = widgets.Button(description = '', disabled = False, button_style = '', tooltip = 'Save measurements', icon = 'save', layout = widgets.Layout(width = '100px', height = '50px'))
quit_button = widgets.Button(description = '', disabled = False, button_style = '', tooltip = 'Main menu', icon = 'home', layout = widgets.Layout(width = '100px', height = '50px'))

bottom_buttons_hbox = widgets.HBox([upload_measurements_button, quit_button])

increase_buttons_size = HTML("""
<style>
    .fa-save {
        font-size: 2em !important; /* Increase icon size */
    }
    .fa-home {
        font-size: 2em !important; /* Increase icon size */
    }
</style>
""")

In [ ]:
# Functions
def read_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

# Function to close the notebook page
def close_notebook(b):
    display(Javascript(f'window.open("home.ipynb", "_blank")'))
    display(Javascript('window.close()'))

def create_measurement_action(b):
    global OPENBIS_SESSION
    
    # samples = OPENBIS_SESSION.get_objects(type = "SAMPLE")
    # samples_names = [sample.props["$name"] for sample in samples]
    
    # if samples_dropdown.value is None:
    #     print("Select a sample.")
    # elif instruments_dropdown.value is None:
    #     print("Select an instrument.")
    # else:
    #     sample_parents = [samples_dropdown.value, instruments_dropdown.value]
    #     object_properties = {}
    #     object_properties["$name"] = measurement_name_textbox.value
    #     object_properties["duration"] = json.dumps({"value": duration_value_floatbox.value, "unit": duration_unit_dropdown.value})
    #     object_properties["pressure"] = json.dumps({"value": pressure_value_floatbox.value, "unit": pressure_unit_dropdown.value})
    #     object_properties["voltage"] = json.dumps({"value": voltage_value_floatbox.value, "unit": voltage_unit_dropdown.value})
    #     object_properties["temperature"] = json.dumps({"value": temperature_value_floatbox.value, "unit": temperature_unit_dropdown.value})
    #     object_properties["current"] = json.dumps({"value": current_value_floatbox.value, "unit": current_unit_dropdown.value})
    #     object_properties["comments"] = comments_textbox.value
        
    #     anneal_task = OPENBIS_SESSION.new_object(type = "ANNEALING", 
    #                                             collection = experiments_dropdown.value,
    #                                             props = object_properties,
    #                                             parents = sample_parents)
    #     anneal_task.save()

    #     OPENBIS_SESSION.new_object(type = "SAMPLE",
    #                             collection = SAMPLES_COLLECTION_OPENBIS_PATH,
    #                             props = {"$name": sample_out_name_textbox.value},
    #                             parents = [anneal_task]).save()
    #     print("Upload successful!")

def load_sample_metadata(change):
    global OPENBIS_SESSION
    if samples_dropdown.value is None:
        sample_details_textbox.value = ''
    else:
        sample_object = OPENBIS_SESSION.get_object(samples_dropdown.value)
        sample_parents_metadata = []
        sample_parents_metadata = get_parents_recursive(sample_object, sample_parents_metadata)
        
        last_sample_process = None
        sample_processes_strings = []
        sample_materials_strings = []
        number_parents = len(sample_parents_metadata)
        parent_idx = 0
        while parent_idx < number_parents:
            parent_metadata = sample_parents_metadata[parent_idx]
            if parent_metadata[0] == "DEPOSITION":
                next_parent_metadata = sample_parents_metadata[parent_idx + 1]
                sample_metadata_string = f"> {parent_metadata[0]} ({parent_metadata[3]}, {parent_metadata[1]}, {parent_metadata[2]}) [{next_parent_metadata[0]} ({next_parent_metadata[3]}, {next_parent_metadata[1]}, {next_parent_metadata[4]})]"
                parent_idx += 1
            else:
                sample_metadata_string = f"> {parent_metadata[0]} ({parent_metadata[3]}, {parent_metadata[1]}, {parent_metadata[2]})"
            
            if parent_metadata[0] in PROCESS_SAMPLE_TYPES:
                sample_processes_strings.append(sample_metadata_string)
                # Get the last sample preparation method performed on the sample in order to search the correct instrument where the sample is being used.
                if last_sample_process is None:
                    last_sample_process = parent_metadata[1]
            else:
                sample_materials_strings.append(sample_metadata_string)
                
            parent_idx += 1
        
        sample_metadata_string = f"PermId: {sample_object.attrs.permId}\nMaterial:"
        
        for material_string in sample_materials_strings:
            sample_metadata_string = f"{sample_metadata_string}\n{material_string}"
            
        sample_metadata_string = f"{sample_metadata_string}\nProcesses:"
        for process_string in sample_processes_strings:
            sample_metadata_string = f"{sample_metadata_string}\n{process_string}"

        if last_sample_process is not None:
            last_sample_process_object = OPENBIS_SESSION.get_object(last_sample_process)
            
            # Automatically select the instrument used in the last sample process task
            last_sample_process_object_parents = last_sample_process_object.parents
            for parent in last_sample_process_object_parents:
                parent_object = OPENBIS_SESSION.get_object(parent)
                if parent_object.type == "INSTRUMENT":
                    instruments_dropdown.value = parent_object.permId
        
        sample_details_textbox.value = sample_metadata_string

def get_parents_recursive(object, object_parents_metadata):
    if object.attrs.type in PROCESS_SAMPLE_TYPES or object.attrs.type in RAW_MATERIALS_TYPES:
        object_parents_metadata.append([object.attrs.type, object.attrs.permId, object.attrs.registrationDate, 
                                        object.props['$name'], object.props['sum_formula']])
    object_parents = object.parents
    for parent in object_parents:
        parent_object = OPENBIS_SESSION.get_object(parent)
        get_parents_recursive(parent_object, object_parents_metadata)
    return object_parents_metadata

# Function to handle file upload and processing
def on_file_upload_change(change):
    with measurements_files_import_output:
        # Clear previous output
        clear_output()
        # Check if a file has been uploaded
        if measurements_files_import.value:
            # Display the name of the uploaded file
            for filename in measurements_files_import.value.keys():
                print(f"Uploaded file: {filename}")
        else:
            print("No file uploaded.")

In [ ]:
# Connect with openBIS
session_data = read_json("token.json")
OPENBIS_SESSION = Openbis(session_data["url"], verify_certificates = False)
OPENBIS_SESSION.set_token(session_data["token"])

# Measurement Uploader

## Select sample

In [ ]:
samples = OPENBIS_SESSION.get_objects(type = "SAMPLE")
instruments = OPENBIS_SESSION.get_objects(type = "INSTRUMENT")

samples_names_permids = [(sample.props['$name'], sample.permId) for sample in samples]
samples_dates = [datetime.fromisoformat(sample.registrationDate) for sample in samples]
samples_names_permids = [x for _,x in sorted(zip(samples_dates, samples_names_permids), reverse = True)]
samples_names_permids.insert(0, ('Select an input sample...', None))
samples_dropdown.options = samples_names_permids

instruments_names_permids = [(f"{instrument.props['$name']} ({instrument.attrs.permId})", instrument.permId) for instrument in instruments]
instruments_names_permids.insert(0, ('Select an instrument...', None))
instruments_dropdown.options = instruments_names_permids
instruments_dropdown.options = instruments_names_permids

display(widgets.VBox([sample_metadata_boxes, instruments_dropdown]))
upload_measurements_button.on_click(create_measurement_action)
quit_button.on_click(close_notebook)
samples_dropdown.observe(load_sample_metadata, names = 'value')
measurements_files_import.observe(on_file_upload_change, names = 'value')

## Upload measurements

In [ ]:
display(measurements_files_import, measurements_files_import_output)
display(bottom_buttons_hbox)
display(increase_buttons_size)